**Setup:**

In [ ]:
%pip install -Uqq fastai torchvision pickle

from fastai.vision.all import *

Importing Data:

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor

train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)

test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True
)

Look at `train_data`:

In [ ]:
train_data

In [ ]:
train_data.data.shape

Create Dataloaders `dl`, `valid_dl` and `dls`:

In [ ]:
dl = DataLoader(train_data, batch_size=100, shuffle=True)
valid_dl = DataLoader(test_data, batch_size=100, shuffle=True)
dls = DataLoaders(dl, valid_dl)

Import `Net` from `learning_functions.py`:

In [ ]:
from learning_functions import Net
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Use GPU if available
neural_net = Net().to(device)

Create `learn` function from `neural_net`:

In [ ]:
from torch import nn
from learning_functions import batch_accuracy

learn = Learner(dls, neural_net, opt_func=SGD, loss_func=nn.CrossEntropyLoss(), metrics=batch_accuracy)

In [ ]:
optimizer = SGD(neural_net.parameters(), lr=0.2)
loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    neural_net.train()
    global optimizer
    for batch_idx, (data, target) in enumerate(dl):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = neural_net(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx*len(data)}/{len(dl.dataset)} ({100. * batch_idx / len(dl):.0f}%)]\t{loss.item():.6f}")

def test():
    neural_net.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in valid_dl:
            data, target = data.to(device), target.to(device)
            output = neural_net(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(valid_dl.dataset)
    print(f'\nTest Loss: Average Loss: {test_loss:.4f}, Accuracy: {correct}/{len(valid_dl.dataset)} {100. * correct / len(valid_dl.dataset):.0f}%')

In [ ]:
from torch import optim
# from learning_functions import train, test

for epoch in range(1, 11):
        train(epoch)
        test()

In [ ]:
learn = Learner(dls, neural_net, opt_func=SGD, loss_func=loss_fn)
learn.fit(5, 0.2)

In [18]:
import pickle

def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
torch.save(neural_net.state_dict(), 'digit_identifier.pth')